# 8. 투표기반 앙상블

## 8.1 핵심 개념

**투표기반 앙상블(Voting Ensemble)은 여러 분류기를 학습 시킨 후 각각의 분류기가 예측하는 레이블의 범주중 가장 높은 예측율 보이는 케이스를 추출**하여 예측값을 생성 하는 방법입니다.

![투표기반앙상블](./extrafiles/voting.png)

투표기반 앙상블은 범주기반의 분류(Hard Leaner와 확률 기반의 분류(Soft Leaner) 모두 가능합니다.

## 8.2 scikit-learn

**투표기반 앙상블은 sklearn.ensemble 패키지**에 속해 있습니다. **분류 알고리즘으로는 VotingClassifier, 회귀분석 알고리즘으로는 VotingRegressor** 가 있습니다.


|sklearn.ensemble|Ensemble Methods|
|:--|:--|
|skleanr.ensemble.RandomForestClassifier() |A random forest classifier. |
|skleanr.ensemble.RandomForestRegressor() |A random forest regressor. |
|skleanr.ensemble.RandomTreeEmbedding() |An ensemble of totally random trees.|
|skleanr.ensemble.StackingClassifier() |Stack of estimators with a final classifier.|
|skleanr.ensemble.StackingRegressor() |Stack of esimators with a final regressor. |
|sklearn.ensemble.VotingClassifier() |Soft Voting/Majority Rule classifier for unfittied estimators. |
|sklearn.ensemble.VotingRegressor() |Prediction voting regressor for unfitted estimators. |
|sklearn.ensemble.HistGradientBoostingRegressor() |Histogram-based Gradient Boosting Regression Tree. |
|sklearn.ensemble.HistGradientBoostingClassifier() |Histogram-based Gradient Boosting Classification Tree. |


**VotingClassifier** 에서의 하이퍼파라미터는 vote 입니다. 범주(hard), 확률(soft) 기반으로 투표를 할지를 결정 합니다.  
**VotingRegressor** 에서는 하이퍼 파라미터가 존재하지 않습니다.

|Hyper Parameter||
|:--|:--|
|Voting|hardm soft (Classifier 에서만)|

## 8.3 분석 코드

### Part1. 분류(Classification)

In [2]:
# 경고레벨조정
import warnings
warnings.filterwarnings("ignore")

# 데이터 로드
import pandas as pd
data = pd.read_csv("./extrafiles/breast-cancer-wisconsin.csv", encoding='utf-8')

# 컬럼정보 확인
print(data.columns)

# 독립변수/ 종속변수 분리
X = data[['Clump_Thickness', 'Cell_Size', 'Cell_Shape',
       'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
       'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses']]
y = data[['Class']]

# train-test data 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# stratify 효과 - 범주형 변수를 유사한 비율로 train / test 데이터로 분리시켜 준다.
print(y_train.mean())
print(y_test.mean())

# 표준화 작업 - MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

Index(['code', 'Clump_Thickness', 'Cell_Size', 'Cell_Shape',
       'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
       'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses', 'Class'],
      dtype='object')
Class    0.349609
dtype: float64
Class    0.350877
dtype: float64


In [3]:
# 투표기반 앙상블
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

logit_model = LogisticRegression(random_state=42)
rnf_model = RandomForestClassifier(random_state=42)
svm_model = SVC(random_state=42)

voting_hard = VotingClassifier(
    estimators=[('lr', logit_model), ('rf', rnf_model), ('svc', svm_model)], voting='hard'
)

voting_hard.fit(X_scaled_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [10]:
# 정확도 계산
from sklearn.metrics import accuracy_score

# 각 알고리즘별 정확도 출력
for clf in (logit_model, rnf_model, svm_model, voting_hard):
    clf.fit(X_scaled_train, y_train)
    y_pred = clf.predict(X_scaled_test)
    print('{:22} : {:.4f}'.format(clf.__class__.__name__, accuracy_score(y_test, y_pred)))

LogisticRegression     : 0.9591
RandomForestClassifier : 0.9649
SVC                    : 0.9649
VotingClassifier       : 0.9649


**[각 모델별 분류 정확도 산출]**

In [5]:
# 로지스틱 회귀모델 혼동행렬 계산
from sklearn.metrics import confusion_matrix
log_pred_train = logit_model.predict(X_scaled_train)
log_confusion_train = confusion_matrix(y_train, log_pred_train)
print("로지스틱 분류기 훈련데이터 오차행렬 : \n", log_confusion_train)

log_pred_test = logit_model.predict(X_scaled_test)
log_confusion_test = confusion_matrix(y_test, log_pred_test)
print("\n로지스틱 분류기 테스트데이터 오차행렬 : \n", log_confusion_test)

로지스틱 분류기 훈련데이터 오차행렬 : 
 [[328   5]
 [  9 170]]

로지스틱 분류기 테스트데이터 오차행렬 : 
 [[106   5]
 [  2  58]]


In [9]:
# SVM 혼동행렬 계산
from sklearn.metrics import confusion_matrix
svm_pred_train = svm_model.predict(X_scaled_train)
svm_confusion_train = confusion_matrix(y_train, svm_pred_train)
print("서포트벡터머신 분류기 훈련데이터 오차행렬 : \n", svm_confusion_train)

svm_pred_test = svm_model.predict(X_scaled_test)
svm_confusion_test = confusion_matrix(y_test, svm_pred_test)
print("\n서포트벡터머신 분류기 테스트데이터 오차행렬 : \n", svm_confusion_test)

서포트벡터머신 분류기 훈련데이터 오차행렬 : 
 [[329   4]
 [  4 175]]

서포트벡터머신 분류기 테스트데이터 오차행렬 : 
 [[106   5]
 [  1  59]]


In [7]:
# Random Forest 혼동행렬 계산
from sklearn.metrics import confusion_matrix
rnf_pred_train = rnf_model.predict(X_scaled_train)
rnf_confusion_train = confusion_matrix(y_train, rnf_pred_train)
print("랜덤포레스트 분류기 훈련데이터 오차행렬 : \n", rnf_confusion_train)

rnf_pred_test = rnf_model.predict(X_scaled_test)
rnf_confusion_test = confusion_matrix(y_test, rnf_pred_test)
print("\n랜덤포레스트 분류기 테스트데이터 오차행렬 : \n", rnf_confusion_test)

랜덤포레스트 분류기 훈련데이터 오차행렬 : 
 [[333   0]
 [  0 179]]

랜덤포레스트 분류기 테스트데이터 오차행렬 : 
 [[106   5]
 [  1  59]]


In [8]:
# voting hard 혼동행렬 계산
from sklearn.metrics import confusion_matrix
voting_pred_train = voting_hard.predict(X_scaled_train)
voting_confusion_train = confusion_matrix(y_train, voting_pred_train)
print("투표 분류기 훈련데이터 오차행렬 : \n", voting_confusion_train)

voting_pred_test = voting_hard.predict(X_scaled_test)
voting_confusion_test = confusion_matrix(y_test, voting_pred_test)
print("\n투표 분류기 테스트데이터 오차행렬 : \n", voting_confusion_test)

투표 분류기 훈련데이터 오차행렬 : 
 [[329   4]
 [  4 175]]

투표 분류기 테스트데이터 오차행렬 : 
 [[106   5]
 [  1  59]]


**[종합정리]**

분석 데이터 에서는 개별 분류 알고리즘과 거의 유사한 결과를 보였습니다만 **일반적으로 앙상블은 개별 알고리즘 조합이 좋으면 좀 더 나은 결과를 보장**합니다. 또 <u>범주(hard) 보다는 확률(soft) 투표 방식이 보다 높은 정확도</u>를 보여줍니다. **실전에 자주 사용되는 방식이므로 반드시 익혀 두어야 합니다.**

### Part2. 회귀(Regression)

In [3]:
# 데이터 로드
data2 = pd.read_csv('./extrafiles/house_price.csv', encoding='utf-8')

print(data2.columns)

X = data2[data2.columns[1:5]]
y = data2[['house_value']]

print(X.columns)

# train-test data 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# stratify 효과 - 범주형 변수를 유사한 비율로 train / test 데이터로 분리시켜 준다.
print(y_train.mean())
print(y_test.mean())

# 표준화 작업 - MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

Index(['housing_age', 'income', 'bedrooms', 'households', 'rooms',
       'house_value'],
      dtype='object')
Index(['income', 'bedrooms', 'households', 'rooms'], dtype='object')
house_value    189260.967812
dtype: float64
house_value    188391.001357
dtype: float64


In [4]:
# 투표기반 앙상블 모델 적용
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor

linear_model = LinearRegression()
rnf_model = RandomForestRegressor(random_state=42)

voting_regressor = VotingRegressor(
    estimators=[('lr', linear_model), ('rf', rnf_model)] # 회귀분석에서는 voting type 이 존재하지 않는다.
)

voting_regressor.fit(X_scaled_train, y_train)

VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('rf', RandomForestRegressor(random_state=42))])

In [6]:
# 훈련 데이터 모델 적용
pred_train = voting_regressor.predict(X_scaled_train)
voting_regressor.score(X_scaled_train, y_train)

0.7962532705428835

In [8]:
# 테스트 데이터 모델 적용
pred_test = voting_regressor.predict(X_scaled_test)
voting_regressor.score(X_scaled_test, y_test)

0.5936371957936408

In [10]:
# 회귀분석의 지표 R Square n RMSE
# RMSE (Root Mean Squared Error)
from sklearn.metrics import mean_squared_error
import numpy as np
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈  련데이터 RMSE:", np.sqrt(MSE_train))
print("테스트데이터 RMSE:", np.sqrt(MSE_test))

훈  련데이터 RMSE: 43082.050654857834
테스트데이터 RMSE: 60942.385243534896


**[종합정리]**

회귀문제에서 투표기반 앙상블로 2개 알고리즘만 조합하였음에도 정확도가 개별 알고리즘을 적용할때 보다 높게 나타났다. 개별 알고리즘의 하이퍼 파라미터를 효과적으로 튜닝한다면 더 좋은 결과도 기대할 수 있다.